La empresa XX, es una empresa dedicada a la venta de productos agrícolas 100%
online. Durante la pandemia se han aumentado significativamente las ventas por
medio de su página web, generando también un incremento elevado en intentos de
fraude al momento de comprar los productos. Por tal motivo se hace necesario contar
con un modelo de machine learning que permita basado en los datos históricos de
transacciones, identificar de manera inmediata el nivel de fraude que pueda tener una
transacción y ser rechazada o aprobada

In [1]:
import pandas as pd
import matplotlib as plt

In [2]:
da = pd.read_csv("D:/Nueva carpeta - copia/transacciones.csv")
da.head()

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Days to deposit,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
0,CO,521715135,2020-08-31 12:41:45.866,8/31/20,8,234543,833883,XYZ-8811992-1598893415162,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,NaN,BANCOLOMBIA,CO,549659******9663,CREDIT,COP,34450,COP,34450.0,MASTERCARD
1,CO,521714842,2020-08-31 12:41:23.219,8/31/20,8,234543,833883,XYZ-8812187-1598894631757,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,NaN,BANCOLOMBIA,CO,540696******7669,CREDIT,COP,45850,COP,45850.0,MASTERCARD
2,CO,521714218,2020-08-31 12:41:00.15,8/31/20,8,234543,833883,XYZ-8812199-1598894873278,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,NaN,BBVA COLOMBIA,CO,553643******9379,CREDIT,COP,50876,COP,50876.0,MASTERCARD
3,CO,521713519,2020-08-31 12:40:19.128,8/31/20,8,234543,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,REFUND,...,NaN,BANCO DAVIVIEND,CO,499635******9266,DEBIT,COP,500,COP,500.0,VISA
4,CO,521713519,2020-08-31 12:40:16.871,8/31/20,8,234543,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,NaN,BANCO DAVIVIEND,CO,499635******9266,DEBIT,COP,500,COP,500.0,VISA


In [3]:
da.info()
#Como primera observación del dataframe, se evidencia valores inexistentes o perdidos en algunas columnas
#algunas con más cantidad que otras como; Bank Response, Message Response Error, Authorization code y Days to deposit, la mayoría
#datos de tipo objeto, es decir que no tienen forma de normalizar númericamente, mientras que hay una cantidad de valores no nulos
#en la columna de Days to Deposit que solo tiene 14 datos presentes
#Igualmente hay algunos datos pérdidos en otras columnas, en baja cantidad como en las columnas: Country, Payment Method
#BIN bank, Country BIN ISO, Visible number, Card type, processing currency y processing value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570312 entries, 0 to 570311
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Country                 570228 non-null  object 
 1   Order Id                570312 non-null  int64  
 2   Creation date           570312 non-null  object 
 3   Fecha                   570312 non-null  object 
 4   mes                     570312 non-null  int64  
 5   Merchant Id             570312 non-null  int64  
 6   Account Id              570312 non-null  int64  
 7   Reference               570312 non-null  object 
 8   Description             570312 non-null  object 
 9   Transaction type        570312 non-null  object 
 10  Status                  570312 non-null  object 
 11  Response code           570312 non-null  object 
 12  Bank Response code      360589 non-null  object 
 13  Message response error  73158 non-null   object 
 14  Payment method      

In [4]:
#Entramos a revisar las columnas con una gran cantidad de datos pérdidos para observar su comportamiento
da.groupby(["Bank Response code"]).count()

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Days to deposit,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
Bank Response code,,,,,,,,,,,,,,,,,,,,,
0,293118,293118,293118,293118,293118,293118,293118,293118,293118,293118,...,6,290137,293093,293118,288688,293118,293118,293118,293118,293118
1,252,252,252,252,252,252,252,252,252,252,...,0,250,252,252,252,252,252,252,252,252
105,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
12,6407,6407,6407,6407,6407,6407,6407,6407,6407,6407,...,0,6400,6407,6407,6407,6407,6407,6407,6407,6407
13,15,15,15,15,15,15,15,15,15,15,...,0,15,15,15,15,15,15,15,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S??lo se puede realizar anulaciones a Transacciones del mismo d??a antes de las 9:00 PM,28,28,28,28,28,28,28,28,28,28,...,0,27,28,28,28,28,28,28,28,28
SUCCESS,9,9,9,9,9,9,9,9,9,9,...,8,0,0,0,0,9,9,9,9,9
"Se present?? un error durante la generaci??n del comprobante, por favor verifique el estado de la transacci??n",1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1


In [5]:
da.groupby(["Message response error"]).count()
#En este caso se puede evidenciar que cada mensaje corresponde a un código de respuesta específico, a lo cual significa
#que la columna Message Response siempre debería corresponder con un valor único dada en la columna Bank Response code

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Days to deposit,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
Message response error,,,,,,,,,,,,,,,,,,,,,
Account not found for card,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
Aprobado,6745,6745,6745,6745,6745,6745,6745,6745,6745,6745,...,3,6694,6745,6745,6659,6745,6745,6745,6745,6745
BIN Inhabilitado,159,159,159,159,159,159,159,159,159,159,...,0,152,159,159,156,159,159,159,159,159
CAF not found,433,433,433,433,433,433,433,433,433,433,...,0,429,433,433,407,433,433,433,433,433
Cardholder state in file (CAF),1498,1498,1498,1498,1498,1498,1498,1498,1498,1498,...,0,1492,1497,1498,1497,1498,1498,1498,1498,1498
Circuit breaker open,14,19,19,19,19,19,19,19,19,19,...,0,19,19,19,18,19,19,0,0,19
Clave Invalida,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
Comercio invalido,70,70,70,70,70,70,70,70,70,70,...,0,0,70,70,70,70,70,70,70,70
Contacte al emisor,252,252,252,252,252,252,252,252,252,252,...,0,250,252,252,252,252,252,252,252,252


In [6]:
da2= da[["Bank Response code", "Message response error", "Valor", "Status"]]
da2.groupby(["Bank Response code"]).count()
#Ahora se puede obervar algunas relaciones, y en otra sub tabla generada se evidencia que el 0 representa el mensaje aprobado
#a lo cuál la mejor manera de rellenar las fillas nulas del Message Error será con la relación de código del Bank response

,Message response error,Valor,Status
Bank Response code,,,
0,6745,293118,293118
1,252,252,252
105,1,1,1
12,6384,6407,6407
13,15,15,15
...,...,...,...
S??lo se puede realizar anulaciones a Transacciones del mismo d??a antes de las 9:00 PM,0,28,28
SUCCESS,0,9,9
"Se present?? un error durante la generaci??n del comprobante, por favor verifique el estado de la transacci??n",0,1,1


In [7]:
da.groupby(["Authorization code"]).count()

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Days to deposit,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
Authorization code,,,,,,,,,,,,,,,,,,,,,
0,51255,51255,51255,51255,51255,51255,51255,51255,51255,51255,...,0,48964,51250,51255,50932,51255,51255,51255,51255,51255
00014B,2,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
00016Z,2,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
00027Z,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
00033C,2,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z25FOH,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
Z9YQNV,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
ZC8534,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1


In [8]:
gt = list(da["Bank Response code"].unique())
print(gt, len(gt))

['0', '51', nan, '12', '5', '62', '14', '6', '4', 'INV04', '98', '59', '41', '57', '87', '56', '36', '54', 'N0', '13', '901', '43', '909', '1', '61', '65', '9004', '903', 'SUCCESS', '911', 'Q1', '31', '83', 'S??lo se puede realizar anulaciones a Transacciones del mismo d??a antes de las 9:00 PM', 'P1', '9008', 'M2', '9009', '206', '58', '3', 'T8', '208', '86', '255', 'INV03', 'T5', '75', '60', '45', '95', '55', 'Se present?? un error durante la generaci??n del comprobante, por favor verifique el estado de la transacci??n', '105', '30', '50', 'INV01', 'Certificado Usuario y/o Clave Inv??lidos', '38', '9002', '33', 'La transacci??n no existe'] 62


In [9]:
seg = list(da["Message response error"].unique())
print(seg, len(seg))

[nan, 'Fondos Insuficientes', 'Invalid transaction', 'Declined, can be locked out card or timeout', 'Restricted card', 'Tarjeta Invalida', 'Aprobado', 'Rechazo General', 'Hold back card', 'Invalid security code', 'Tarjeta Perdida', 'Transacci?n No Permitida', 'Tarjeta Restringida', 'Transaction not allowed to credit card', 'Expired card', 'BIN Inhabilitado', 'Monto Invalido', 'Stolen or lost card', 'Tarjeta Vencida', 'No response from acquirer', 'Expired card with retention order', 'Cardholder state in file (CAF)', 'Transacci?n Invalida', 'Contacte al emisor', 'Excede Monto Limite', 'Usage limit exceeded for current period', 'Error tecnico del servicio', 'ERROR NO PARAMETRIZADO', 'No hay rutas activas para procesar la autorizaci?n', 'Transaction processing was not possible', 'Tarjeta Perdida o Robada', 'Problema de Enrutamiento', 'Invalid card state', 'Original no Encontrado', 'Datos de autenticaci?n inv?lidos', 'CAF not found', 'Comercio invalido', 'Invalid expiration date', 'Ground l

In [10]:
da["Days to deposit"].unique()
#Esto explica la razón por la que esta columna solo presenta 14 datos no nulos, a los cual solo se diferencia un 0 los datos
#existentes en el dataframe, en este caso debido a que no existe ninguna relación entre ambos datos y el resto de las columnas
#esta variable se hace innecesaria para el análisis

array([nan,  0.])

In [11]:
da3 = da[["Order Id","Status", "Bank Response code", "Message response error", "Authorization code", "Days to deposit"]]
da3[da3["Days to deposit"].notnull()]
#Este filtro fuera de tratar de encontrar alguna diferencia de relación entre los datos de Days to deposit
#Nos ayuda a determinar que existe también inconsistencias de estandarización entre las columnas de Response Code, y Message Error
#así mismo en cuanto a estas 2 columnas lo mejor es mantener el bank response code dentro del tratamiento para evitar las 
#inconsistencias con su columna de referencia

,Order Id,Status,Bank Response code,Message response error,Authorization code,Days to deposit
653,521572551,APPROVED,0,Aprobado,163759,0.0
13947,517148215,APPROVED,SUCCESS,NaN,NaN,0.0
18907,515824728,APPROVED,0,Aprobado,96268Z,0.0
32241,512829412,APPROVED,0,NaN,4781,0.0
34011,512418718,APPROVED,SUCCESS,NaN,NaN,0.0
37036,511889425,APPROVED,0,NaN,06541B,0.0
41218,511771299,APPROVED,0,Aprobado,02498P,0.0
44971,519883833,APPROVED,SUCCESS,NaN,NaN,0.0
56045,517514717,APPROVED,SUCCESS,NaN,NaN,0.0
56696,517318758,APPROVED,SUCCESS,NaN,NaN,0.0


In [12]:
da2 = da[["Order Id", "Status", "Bank Response code", "Message response error", "Authorization code", "Days to deposit", "Valor"]]
da2[da2["Message response error"].notnull()]

,Order Id,Status,Bank Response code,Message response error,Authorization code,Days to deposit,Valor
1,521714842,DECLINED,51,Fondos Insuficientes,0,NaN,45850
10,521711193,DECLINED,12,Invalid transaction,123753,NaN,500
42,521888884,DECLINED,5,"Declined, can be locked out card or timeout",619648,NaN,500
53,521884139,DECLINED,62,Restricted card,ND,NaN,20050
67,521882188,DECLINED,12,Invalid transaction,0,NaN,500
...,...,...,...,...,...,...,...
570268,513711498,DECLINED,59,Restricted card,0,NaN,115430
570276,513711134,DECLINED,14,Tarjeta Invalida,0,NaN,60710
570290,513711851,DECLINED,62,Restricted card,0,NaN,9125
570308,513712578,DECLINED,62,Restricted card,0,NaN,20050


In [13]:
da3 = da[["Order Id", "Status", "Bank Response code", "Message response error", "Authorization code", "Days to deposit", "Valor"]]
da3[da3["Authorization code"].isnull()]
#Analizando comportamientos de las anteriores columnas, se evidencia que los valores "pérdidos" en la columna de Authorization Code
#Representa aquellas transaccines declinadas en la operación, sin embargo al generarse un código único por transacción
#aprobada este no se debería considerar como variable útil debido a su gran carga en el procesamiento
#no obstante no significa que este sea el único valor que representa transacciones declinadas, pues este caso también se presenta
#como 0 y otra referencia de Message Error y Bank Response code

,Order Id,Status,Bank Response code,Message response error,Authorization code,Days to deposit,Valor
2,521714218,DECLINED,NaN,NaN,NaN,NaN,50876
40,521887429,DECLINED,NaN,NaN,NaN,NaN,849600
52,1184588732,DECLINED,NaN,NaN,NaN,NaN,73155
54,521883751,DECLINED,NaN,NaN,NaN,NaN,157530
57,521883598,DECLINED,NaN,NaN,NaN,NaN,567150
...,...,...,...,...,...,...,...
570304,513712358,DECLINED,NaN,NaN,NaN,NaN,62310
570305,513712327,DECLINED,NaN,NaN,NaN,NaN,135930
570306,513712382,DECLINED,NaN,NaN,NaN,NaN,11690
570307,513712424,DECLINED,NaN,NaN,NaN,NaN,24515


In [14]:
da4 = da[["Order Id", "Status", "Bank Response code", "Message response error", "Authorization code", "Days to deposit", "Valor"]]
da4[da4["Authorization code"].notnull()]

,Order Id,Status,Bank Response code,Message response error,Authorization code,Days to deposit,Valor
0,521715135,APPROVED,0,NaN,R04292,NaN,34450
1,521714842,DECLINED,51,Fondos Insuficientes,0,NaN,45850
3,521713519,APPROVED,0,NaN,20510,NaN,500
4,521713519,APPROVED,0,NaN,18063,NaN,500
5,521713341,APPROVED,0,NaN,124001,NaN,46950
...,...,...,...,...,...,...,...
570276,513711134,DECLINED,14,Tarjeta Invalida,0,NaN,60710
570290,513711851,DECLINED,62,Restricted card,0,NaN,9125
570308,513712578,DECLINED,62,Restricted card,0,NaN,20050
570309,513714958,APPROVED,0,NaN,525905,NaN,65400


In [15]:
da["Status"].unique()
#La columna Status resume en mejor medida las categorías existentes en las transacciones, a lo cual representa mayor importancia
#que las otras 2 columnas que se relacionan que la misma actividad, así mismo se ha evidenciado que no hay suficiente estandarización
#en la columna de Bank Response Code, pues no sólo hay códigos sino mensajes concretos y códigos fuera de los comunes
#esto impide una normalización sencilla de la información, por ello tal columna no se tomará en el tratamiento y se resumirán,
#como antes mencionado, bajo la columna Status

array(['APPROVED', 'DECLINED', 'ERROR'], dtype=object)

In [16]:
#Entonces considerando los hallazgos anteriores, con gran cantidad de datos pérdidos en varias columnas, de ver que tienen relación
#con otra columna más estandarizada y evitando una masa innecesaria de información hasta el momento se ha podido reducir el dataframe
#de la siguiente forma

daf1 = da.drop(["Bank Response code", "Message response error", "Authorization code","Days to deposit"], axis = 1)
daf1

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Accreditation model,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
0,CO,521715135,2020-08-31 12:41:45.866,8/31/20,8,234543,833883,XYZ-8811992-1598893415162,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCOLOMBIA,CO,549659******9663,CREDIT,COP,34450,COP,34450.0,MASTERCARD
1,CO,521714842,2020-08-31 12:41:23.219,8/31/20,8,234543,833883,XYZ-8812187-1598894631757,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCOLOMBIA,CO,540696******7669,CREDIT,COP,45850,COP,45850.0,MASTERCARD
2,CO,521714218,2020-08-31 12:41:00.15,8/31/20,8,234543,833883,XYZ-8812199-1598894873278,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BBVA COLOMBIA,CO,553643******9379,CREDIT,COP,50876,COP,50876.0,MASTERCARD
3,CO,521713519,2020-08-31 12:40:19.128,8/31/20,8,234543,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,REFUND,...,IMMEDIATE,BANCO DAVIVIEND,CO,499635******9266,DEBIT,COP,500,COP,500.0,VISA
4,CO,521713519,2020-08-31 12:40:16.871,8/31/20,8,234543,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCO DAVIVIEND,CO,499635******9266,DEBIT,COP,500,COP,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,CO,513712424,2020-08-11 00:13:22.733,8/11/20,8,234543,839429,XYZ-8664994-1597123036915,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANK OF AMERICA,US,536260******9660,DEBIT,COP,24515,COP,24515.0,MASTERCARD
570308,CO,513712578,2020-08-11 00:13:54.683,8/11/20,8,234543,839429,XYZ-8664996-1597122889687,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCOLOMBIA,CO,540696******6007,CREDIT,COP,20050,COP,20050.0,MASTERCARD
570309,CO,513714958,2020-08-11 00:21:28.031,8/11/20,8,234543,833883,XYZ-8664878-1597120977438,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCO AV VILLAS,CO,547646******6563,CREDIT,COP,65400,COP,65400.0,MASTERCARD
570310,CO,513712137,2020-08-11 00:48:47.001,8/11/20,8,234543,833883,XYZ-8664885-1597121705373,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCOLOMBIA,CO,409993******2909,CREDIT,COP,140875,COP,140875.0,VISA


Ahora se revisará la información de las columnas que presentan una baja cantidad de datos pérdidos con respecto a la cantidad general de datos del dataframe original.
Recordando esta pequeñas pérdidas de información se encuentran las columnas: Country, Payment Method
#BIN bank, Country BIN ISO, Visible number, Card type, processing currency y processing value

In [17]:
#Comenzando con la columna de Country se evidencia que al parecer no se ingresó la información correcta en el momento indicado
#con respecto al país de origen de la transacción, en relación se observa que el BIN ISO para esos datos corresponde al mismo
#indicado en los valores no nulos, lo cual tembién permite conocer que el país de origen es el mismo, y no sería, a su vez, una
#variable que afecte a las demás debido su valor único
db=daf1[daf1.Country.isnull()]
db

,Country,Order Id,Creation date,Fecha,mes,Merchant Id,Account Id,Reference,Description,Transaction type,...,Accreditation model,BIN Bank,Country BIN ISO,Visible number,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
21470,NaN,515179978,2020-08-24 19:21:48.271,8/24/20,8,234543,839429,XYZ-104936-1598314908027,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******7262,CREDIT,COP,500,NaN,NaN,CODENSA
22911,NaN,514837283,2020-08-24 11:56:55.81,8/24/20,8,234543,839429,XYZ-604075-1598288215575,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******6775,CREDIT,COP,500,NaN,NaN,CODENSA
43639,NaN,511171918,2020-08-18 19:27:05.64,8/18/20,8,234543,839429,XYZ-53769-1597796825352,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******6995,CREDIT,COP,500,NaN,NaN,CODENSA
69112,NaN,513895581,2020-08-11 09:36:16.904,8/11/20,8,234543,839429,XYZ-293422-1597156576618,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******6232,CREDIT,COP,500,NaN,NaN,CODENSA
134797,NaN,412885924,2020-04-03 14:14:00.108,4/3/20,4,234543,833883,XYZ-7609540-1585940055288,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,BANCO FALABELLA,CO,529209******4627,CREDIT,COP,53650,NaN,NaN,MASTERCARD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549063,NaN,497458713,2020-08-03 14:21:50.747,8/3/20,8,234543,839429,XYZ-62805-1596482509957,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******7049,CREDIT,COP,500,NaN,NaN,CODENSA
561468,NaN,511814739,2020-08-08 14:01:07.071,8/8/20,8,234543,839429,XYZ-542224-1596913266730,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******6904,CREDIT,COP,500,NaN,NaN,CODENSA
566107,NaN,512851994,2020-08-09 17:23:49.809,8/9/20,8,234543,839429,XYZ-707363-1597011829553,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******0066,CREDIT,COP,500,NaN,NaN,CODENSA
566114,NaN,512851423,2020-08-09 17:24:23.614,8/9/20,8,234543,839429,XYZ-707363-1597011863356,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,...,IMMEDIATE,COLPATRIA RED MULTIBANCA,CO,590762******0066,CREDIT,COP,500,NaN,NaN,CODENSA


In [18]:
daf1a = daf1[["Order Id", "Status","Valor", "Payment method", "BIN Bank", "Country BIN ISO", "Visible number", "Card type", "Processing currency", "Processing value"]]
daf1a[daf1a["Payment method"].isnull()]
#En este caso se evidenci que el valor nulo de la columna de método de pago, al perecer se ve generado por un error en la transacción
#también cabe mencionar que entre las columnas mencionadas, esta no presenta tantos datos pérdidos como los demás

,Order Id,Status,Valor,Payment method,BIN Bank,Country BIN ISO,Visible number,Card type,Processing currency,Processing value
21470,515179978,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******7262,CREDIT,NaN,NaN
22911,514837283,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******6775,CREDIT,NaN,NaN
43639,511171918,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******6995,CREDIT,NaN,NaN
69112,513895581,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******6232,CREDIT,NaN,NaN
134797,412885924,ERROR,53650,NaN,BANCO FALABELLA,CO,529209******4627,CREDIT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
549063,497458713,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******7049,CREDIT,NaN,NaN
561468,511814739,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******6904,CREDIT,NaN,NaN
566107,512851994,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******0066,CREDIT,NaN,NaN
566114,512851423,ERROR,500,NaN,COLPATRIA RED MULTIBANCA,CO,590762******0066,CREDIT,NaN,NaN


In [19]:
daf1b = daf1[["Order Id", "Merchant Id", "Account Id", "Status","Valor", "Payment method", "BIN Bank", "Country BIN ISO", "Visible number", "Card type", "Processing currency"]]
daf1b[daf1b["BIN Bank"].isnull()]

,Order Id,Merchant Id,Account Id,Status,Valor,Payment method,BIN Bank,Country BIN ISO,Visible number,Card type,Processing currency
52,1184588732,234543,839429,DECLINED,73155,CRED_VISA,NaN,NaN,432346******9596,NaN,NaN
110,521877882,234543,839429,DECLINED,188485,CRED_VISA,NaN,US,435665******7569,CREDIT,COP
133,521875458,234543,839429,DECLINED,41460,REDEBAN_VISA,NaN,RO,476603******0299,CREDIT,COP
275,1184585578,234543,833883,DECLINED,84950,CRED_VISA,NaN,NaN,456307******6670,NaN,NaN
297,1184584782,234543,833883,DECLINED,84950,CRED_VISA,NaN,NaN,456307******6670,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
569887,513582924,234543,839429,DECLINED,188485,CRED_VISA,NaN,US,435665******7569,CREDIT,COP
570128,513852841,234543,839429,DECLINED,44850,REDEBAN_VISA,NaN,RO,476603******0299,CREDIT,COP
570144,513853938,234543,839429,DECLINED,188485,CRED_VISA,NaN,US,435665******7569,CREDIT,COP
570257,513899958,234543,839429,DECLINED,44850,REDEBAN_VISA,NaN,RO,476603******0299,CREDIT,COP


In [20]:
print("Country:", daf1["Country"].unique(), "Payment method:", daf1["Payment method"].unique(),
    "Country BIN ISO", daf1["Country BIN ISO"].unique(), 
     "Card type:", daf1["Card type"].unique(), "Processing currency:", daf1["Processing currency"].unique())

Country: ['CO' nan] Payment method: ['CRED_MASTERCARD' 'REDEBAN_MASTERCARD' 'CRED_VISA' 'CRED_DINERS'
 'REDEBAN_VISA' 'CRED_AMEX' 'REDEBAN_DINERS' 'REDEBAN_AMEX' 'PSE' nan
 'CRED_VISA_DEBIT'] Country BIN ISO ['CO' 'US' 'HK' nan 'HN' 'BO' 'CL' 'BR' 'SV' 'AR' 'TM' 'PT' 'RO' 'CH' 'TW'
 'MX' 'IT' 'GB' 'CN' 'EC' 'AU' 'VE' 'PE' 'PA' 'CA' 'UY' 'DO' 'DE' 'GT'
 'BE' 'PR' 'UA' 'ES' 'KY' 'KR' 'FI' 'MY' 'CR' 'IN' 'CW' 'RS' 'IE' 'FR'
 'JP' 'SG' 'SA' 'NO' 'DK' 'NI' 'GA' 'EG' 'IQ' 'ZA' 'DZ' 'NZ' 'AT' 'NL'
 'AE' 'GI' 'PH' 'MV' 'PL' 'IL' 'AD' 'LB' 'UG' 'NG' 'EE' 'SK' 'QA' 'TR'
 'KE' 'PF' 'BN' 'HU' 'ID' 'BH' 'PY' 'BM' 'RU' 'TN' 'MA' 'MU' 'KH' 'AO'
 'BD' 'OM' 'TH' 'PK' 'BA' 'LV' 'SE' 'TT' 'CZ' 'GE' 'LC' 'PS' 'AW' 'KG'
 'BZ' 'MW' 'TZ' 'NP' 'SB' 'AG' 'LT' 'GR' 'ZW'] Card type: ['CREDIT' 'DEBIT' nan] Processing currency: ['COP' nan]


In [21]:
print("BIN Bank:", list(daf1["BIN Bank"].unique()), len(list(daf1["BIN Bank"].unique())))

BIN Bank: ['BANCOLOMBIA', 'BBVA COLOMBIA', 'BANCO DAVIVIEND', 'BANCO DAVIVIENDA', 'BANCO AV VILLAS', 'SCOTIABANK', 'BANCO DE OCCIDENTE', 'BANCO FALABELLA', 'FIRST', 'ITAU', 'RIPLEY COMPANIA DE FINANCIAMIENTO', 'STANDARD CHARTERED', 'NEQUI', 'CITIBANK', nan, 'BANCO DE BOGOTA', 'BANK OF AMERICA', 'BANCO DE DESARROLLO RURAL HONDURAS S.A.', 'TDECU', 'BANCO NACIONAL DE BOLIVIA', 'THINK', 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'FIRST FINANCIAL FEDERAL CREDIT UNION', 'SCOTIABANK MASTERCARD', 'COLPATRIA RED MULTIBANCA', 'ROYAL CREDIT UNION', 'METABANK', 'BANCO CITIBANK DE EL SALVADOR S.A.', 'CHASE', 'PAYPAL', 'BANCO DE GALICIA Y BUENOS AIRES, S.A.', 'STATE BANK FOR FOREIGN ECONOMIC AFFAIRS OF TURKMENISTAN', 'CITI', 'NAVY FEDERAL CREDIT UNION', 'CAIXA GERAL DE DEPOSITOS SA', 'FIFTH THIRD BANK, THE', 'CREDIT SUISSE', 'HSBC', 'CITIZENS', "STATE EMPLOYEES' CREDIT UNION", 'WELLS FARGO', 'HANCOCK WHITNEY', 'SHIN KONG', 'BANCO CAJA SOCIAL BCSC', 'BANCOMER', 'BANCO CITIBANK DE HONDURAS S.A.', 'FIA C

In [22]:
daf1["Merchant Id"].unique()

array([234543], dtype=int64)

In [23]:
daf1["Account Id"].unique()

array([833883, 839429, 838923, 795309], dtype=int64)

No obstante, debido a la diversidad de valores presentes en cada una de estas columnas, no ideal eliminar los renglones en blanco, por tanto para estas columnas lo mejor es ingresar un valor diferente en representación no información no especificada, esto depués de determinar cuáles de estas variables serán o no útiles en el análisis

Así mismo se evidencia por la columna Merchant Id que todas las ventas de realizan en el mismo sitio, lo cuál tampoco afectaría en gran medida con las demás variables.

Otro elemento importante es la columna de Visible number, si bien podría ser un identificador clave de las personas que están realizando la transacción, no hay un elemento que garantice unicidad en el caso, a lo cual considerando la columna Account Id y en lo posible la columna Descriprion es suficiente para determinar el origen de la transacción por parte del cliente. No sin mencionar las diversas variables que contemplan las entidades bancarias y de asociación de las tarjetas utilizadas.


In [24]:
daf2 = daf1.drop(["Country", "Merchant Id", "Visible number"], axis = 1)
daf2

,Order Id,Creation date,Fecha,mes,Account Id,Reference,Description,Transaction type,Status,Response code,...,Transaction origin,Accreditation model,BIN Bank,Country BIN ISO,Card type,Transaction currency,Valor,Processing currency,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8/31/20,8,833883,XYZ-8811992-1598893415162,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,...,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,COP,34450,COP,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8/31/20,8,833883,XYZ-8812187-1598894631757,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,...,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,COP,45850,COP,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8/31/20,8,833883,XYZ-8812199-1598894873278,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,...,POST_API,IMMEDIATE,BBVA COLOMBIA,CO,CREDIT,COP,50876,COP,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8/31/20,8,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,REFUND,APPROVED,APPROVED,...,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,COP,500,COP,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8/31/20,8,839429,XYZ-464504-1598895616616,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,...,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,COP,500,COP,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8/11/20,8,839429,XYZ-8664994-1597123036915,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,...,POST_API,IMMEDIATE,BANK OF AMERICA,US,DEBIT,COP,24515,COP,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8/11/20,8,839429,XYZ-8664996-1597122889687,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,...,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,COP,20050,COP,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8/11/20,8,833883,XYZ-8664878-1597120977438,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,...,POST_API,IMMEDIATE,BANCO AV VILLAS,CO,CREDIT,COP,65400,COP,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8/11/20,8,833883,XYZ-8664885-1597121705373,XYZ agradece su compra.,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,...,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,COP,140875,COP,140875.0,VISA


In [25]:
daf2["Transaction currency"].unique()

array(['COP'], dtype=object)

In [26]:
daf2["Processing currency"].unique()

array(['COP', nan], dtype=object)

In [27]:
daf2["mes"].unique()

array([8, 6, 5, 4, 3, 7], dtype=int64)

Considerando las columnas analizadas en renglones anteriores la única columna valida a mantener entre estas es la de mes, pues identifica con claridad el periodo en el que se ralizaron cada una de las transacciones mencionadas en el dataset, así mismo como se aclara al comienzo del ejercicio, son del mismo año, por lo cuál ignorar la columna de fecha no representaría variación importante en el análisis. En cuando al processing currency se ven datos nulos en su columna, sin embargo la gran mayoría representa a la moneda colombina como el currency utilizado en las transacción, y un filtrado del mismo demuestra que los datso nulos corresponden a transacciones declinadas, otras columnas representan mayor utilidad para este objetivo.
En otra información, la columna Description, pordría ser útil para determinar el origen de las transacciones al existir gran cantidad de información no útil para modelo de análisis ML, tampoco será útil incluirlo en el procedimiento.

In [28]:
daf2a = daf2[["Order Id", "Status","Valor", "Card type","Transaction currency","Processing currency"]]
daf2a[daf2a["Card type"].isnull()]

,Order Id,Status,Valor,Card type,Transaction currency,Processing currency
52,1184588732,DECLINED,73155,NaN,COP,NaN
275,1184585578,DECLINED,84950,NaN,COP,NaN
297,1184584782,DECLINED,84950,NaN,COP,NaN
364,1184553489,DECLINED,233590,NaN,COP,NaN
526,1184552281,DECLINED,233590,NaN,COP,NaN
...,...,...,...,...,...,...
568912,1183124118,DECLINED,157450,NaN,COP,NaN
568998,1183128884,DECLINED,67070,NaN,COP,NaN
569192,1183134858,DECLINED,90745,NaN,COP,NaN
569202,1183135287,DECLINED,90745,NaN,COP,NaN


In [29]:
daf3 = daf2.drop(["Transaction currency", "Processing currency", "Description", "Reference"], axis=1)
daf3

,Order Id,Creation date,Fecha,mes,Account Id,Transaction type,Status,Response code,Payment method,Payment model,Transaction origin,Accreditation model,BIN Bank,Country BIN ISO,Card type,Valor,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8/31/20,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,34450,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8/31/20,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,45850,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8/31/20,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BBVA COLOMBIA,CO,CREDIT,50876,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8/31/20,8,839429,REFUND,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8/31/20,8,839429,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8/11/20,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANK OF AMERICA,US,DEBIT,24515,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8/11/20,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,20050,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8/11/20,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCO AV VILLAS,CO,CREDIT,65400,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8/11/20,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,140875,140875.0,VISA


En esta etapa se evidencia que las columnas de Fecha y mes provienen de la columna de Creation Date, lo cuál representa datos duplicados, no obstante, en este caso como parte de la limpieza del dataset, se eliminará únicamente la columna de fecha

In [30]:
daf4 = daf3.drop(["Fecha"], axis =1)
daf4

,Order Id,Creation date,mes,Account Id,Transaction type,Status,Response code,Payment method,Payment model,Transaction origin,Accreditation model,BIN Bank,Country BIN ISO,Card type,Valor,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,34450,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,45850,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BBVA COLOMBIA,CO,CREDIT,50876,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8,839429,REFUND,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8,839429,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANK OF AMERICA,US,DEBIT,24515,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,20050,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCO AV VILLAS,CO,CREDIT,65400,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,140875,140875.0,VISA


In [31]:
daf4.info()
#Una vez eliminadas columnas que no representan valor fundamental para el análisis, regresamos a observar si existen datos pérdidos
#en el dataset, para realizar observación y rellenar en lo posible tales resultados. Algunos de estos valores se ven en el renglon 20

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570312 entries, 0 to 570311
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Order Id             570312 non-null  int64  
 1   Creation date        570312 non-null  object 
 2   mes                  570312 non-null  int64  
 3   Account Id           570312 non-null  int64  
 4   Transaction type     570312 non-null  object 
 5   Status               570312 non-null  object 
 6   Response code        570312 non-null  object 
 7   Payment method       570228 non-null  object 
 8   Payment model        570312 non-null  object 
 9   Transaction origin   570312 non-null  object 
 10  Accreditation model  566005 non-null  object 
 11  BIN Bank             559849 non-null  object 
 12  Country BIN ISO      566932 non-null  object 
 13  Card type            561906 non-null  object 
 14  Valor                570312 non-null  int64  
 15  Processing value 

In [32]:
daf4a = daf4[["Order Id", "Status","Valor", "Card type","Accreditation model", "Country BIN ISO"]]
daf4a[daf4a["Country BIN ISO"].isnull()]

,Order Id,Status,Valor,Card type,Accreditation model,Country BIN ISO
52,1184588732,DECLINED,73155,NaN,NaN,NaN
275,1184585578,DECLINED,84950,NaN,NaN,NaN
297,1184584782,DECLINED,84950,NaN,NaN,NaN
364,1184553489,DECLINED,233590,NaN,NaN,NaN
526,1184552281,DECLINED,233590,NaN,NaN,NaN
...,...,...,...,...,...,...
568912,1183124118,DECLINED,157450,NaN,NaN,NaN
568998,1183128884,DECLINED,67070,NaN,NaN,NaN
569192,1183134858,DECLINED,90745,NaN,NaN,NaN
569202,1183135287,DECLINED,90745,NaN,NaN,NaN


In [33]:
daf4["Accreditation model"].unique()

array(['IMMEDIATE', nan], dtype=object)

In [34]:
daf4["Payment method"].fillna(value = "Error", inplace = True)
daf4["Accreditation model"].fillna(value = "Not specified Declined", inplace = True)
daf4["Country BIN ISO"].fillna(value = "Not specified", inplace = True)
daf4["Card type"].fillna(value = "Not specified", inplace = True)
daf4["Processing value"].fillna(value = 0, inplace = True)

daf4

,Order Id,Creation date,mes,Account Id,Transaction type,Status,Response code,Payment method,Payment model,Transaction origin,Accreditation model,BIN Bank,Country BIN ISO,Card type,Valor,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,34450,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,45850,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BBVA COLOMBIA,CO,CREDIT,50876,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8,839429,REFUND,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8,839429,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCO DAVIVIEND,CO,DEBIT,500,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANK OF AMERICA,US,DEBIT,24515,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,20050,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,BANCO AV VILLAS,CO,CREDIT,65400,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,BANCOLOMBIA,CO,CREDIT,140875,140875.0,VISA


In [35]:
daf4["BIN Bank"].fillna(value = "Not specified", inplace = True)
daf4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570312 entries, 0 to 570311
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Order Id             570312 non-null  int64  
 1   Creation date        570312 non-null  object 
 2   mes                  570312 non-null  int64  
 3   Account Id           570312 non-null  int64  
 4   Transaction type     570312 non-null  object 
 5   Status               570312 non-null  object 
 6   Response code        570312 non-null  object 
 7   Payment method       570312 non-null  object 
 8   Payment model        570312 non-null  object 
 9   Transaction origin   570312 non-null  object 
 10  Accreditation model  570312 non-null  object 
 11  BIN Bank             570312 non-null  object 
 12  Country BIN ISO      570312 non-null  object 
 13  Card type            570312 non-null  object 
 14  Valor                570312 non-null  int64  
 15  Processing value 

Ahora sí obtenemos una dataset limpio con ningún valor pérdido en el procedimiento con los datos de mayor interés e importancia para el análisis

Ahora, si bien este dataset ya está limpio, debido a la gran cantidad de información difícil de estandarizar en la columna de BIN Bank, se eliminará para el modelo de tratamiento por ML, pues se crea el anterior archivo para crear visualizaciones con otra herramienta.
Y al mismo tiempo se harán transformaciones pertinentes en la mayoría de variables tipo objeto para obtener mejores resultados

In [36]:
daf5 = daf4.drop(["BIN Bank"], axis=1)
daf5

,Order Id,Creation date,mes,Account Id,Transaction type,Status,Response code,Payment method,Payment model,Transaction origin,Accreditation model,Country BIN ISO,Card type,Valor,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,34450,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,45850,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,50876,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8,839429,REFUND,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,DEBIT,500,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8,839429,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,DEBIT,500,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,US,DEBIT,24515,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,20050,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,65400,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,140875,140875.0,VISA


Ahora se viene la transformación de datos, para aquellas variables tipo objeto que no presentan más de 4 valores únicos, se utilizará la función get_dummies de pandas, con el fin de crear vectores de identificación binaria para estos

In [37]:
print("Transaction type:", daf5["Transaction type"].unique(), ". Status:", daf5["Status"].unique(), ". Response code:", daf5["Response code"].unique(),
     ". Payment method:", daf5["Payment method"].unique(), ". Payment model:", daf5["Payment model"].unique(),
     ". Transaction origin:", daf5["Transaction origin"].unique(), ". Accreditation model:", daf5["Accreditation model"].unique(),
     ". Card type:", daf5["Card type"].unique(), ". Franchise:", daf5["Franchise"].unique())

Transaction type: ['AUTHORIZATION_AND_CAPTURE' 'REFUND' 'VOID'] . Status: ['APPROVED' 'DECLINED' 'ERROR'] . Response code: ['APPROVED' 'INSUFFICIENT_FUNDS' 'ANTIFRAUD_REJECTED'
 'INVALID_TRANSACTION' 'ENTITY_DECLINED' 'RESTRICTED_CARD'
 'INVALID_EXPIRATION_DATE_OR_SECURITY_CODE' 'PAYMENT_NETWORK_REJECTED'
 'INVALID_CARD' 'EXPIRED_CARD' 'REPEAT_TRANSACTION' 'EXCEEDED_AMOUNT'
 'INACTIVE_PAYMENT_PROVIDER' 'CONTACT_THE_ENTITY'
 'ERROR_FIXING_AND_REVERSING' 'INTERNAL_PAYMENT_PROVIDER_ERROR' 'ERROR'
 'BANK_UNREACHABLE' 'NOT_ACCEPTED_TRANSACTION'
 'PAYMENT_NETWORK_NO_CONNECTION' 'PAYMENT_NETWORK_BAD_RESPONSE'
 'ENTITY_MESSAGING_ERROR' 'FIX_NOT_REQUIRED'
 'AUTOMATICALLY_FIXED_AND_SUCCESS_REVERSAL' 'NOT_FIXED_FOR_ERROR_STATE'
 'PAYMENT_NETWORK_NO_RESPONSE'] . Payment method: ['CRED_MASTERCARD' 'REDEBAN_MASTERCARD' 'CRED_VISA' 'CRED_DINERS'
 'REDEBAN_VISA' 'CRED_AMEX' 'REDEBAN_DINERS' 'REDEBAN_AMEX' 'PSE' 'Error'
 'CRED_VISA_DEBIT'] . Payment model: ['GATEWAY' 'AGGREGATOR'] . Transaction origin:

In [38]:
#Dada una observación anterior, se realizará una observación gráfica de la variable Status con ayuda de la libreria Seaborn
daf5.groupby(["Status"]).count()

,Order Id,Creation date,mes,Account Id,Transaction type,Response code,Payment method,Payment model,Transaction origin,Accreditation model,Country BIN ISO,Card type,Valor,Processing value,Franchise
Status,,,,,,,,,,,,,,,
APPROVED,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123,293123
DECLINED,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892,276892
ERROR,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297


Tal como se aprecia en la línea anterios en la variable Status, existe un valor específico con una cantidad baja de observaciones con base en la cantidad total del datafram, que es el valor ERROR, el cual con el fin de conocer qué otra información arrojan las demás columnas se realizará un nuevo filtro con base en este para observar la utilidad de esta dentro del tratamiento final

In [39]:
daf5a = daf5[["Order Id", "Status","Valor", "Response code", "Card type","Franchise", "Processing value"]]
daf5a[daf5a["Status"]=="ERROR"]

,Order Id,Status,Valor,Response code,Card type,Franchise,Processing value
11850,517878133,ERROR,60710,ERROR_FIXING_AND_REVERSING,CREDIT,VISA,60710.0
11874,517875845,ERROR,38640,ERROR_FIXING_AND_REVERSING,CREDIT,MASTERCARD,38640.0
11876,517875827,ERROR,40100,ERROR_FIXING_AND_REVERSING,CREDIT,VISA,40100.0
21470,515179978,ERROR,500,ERROR,CREDIT,CODENSA,0.0
22911,514837283,ERROR,500,ERROR,CREDIT,CODENSA,0.0
...,...,...,...,...,...,...,...
549063,497458713,ERROR,500,ERROR,CREDIT,CODENSA,0.0
561468,511814739,ERROR,500,ERROR,CREDIT,CODENSA,0.0
566107,512851994,ERROR,500,ERROR,CREDIT,CODENSA,0.0
566114,512851423,ERROR,500,ERROR,CREDIT,CODENSA,0.0


Observando este comportamiento, nos podemos das cuenta que las transacciones que marcan error, no generan un código de respuesta distinto al mencionado, y en caso tal de que haya un monto, al parecer específico o alto, el sistema genera una devolución. Dados estos aspectos y que con respecto el total de filas, este resultado no es significativo, se eliminarán del modelo las filas del dataset cuyo Status sea error

In [40]:
daf5b = daf5.drop(daf5[daf5["Status"]=="ERROR"].index)
daf5b
#daf5b.to_csv("Antifraud1.csv")
#daf5b.to_excel("Antifraud1.xlsx")

,Order Id,Creation date,mes,Account Id,Transaction type,Status,Response code,Payment method,Payment model,Transaction origin,Accreditation model,Country BIN ISO,Card type,Valor,Processing value,Franchise
0,521715135,2020-08-31 12:41:45.866,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,34450,34450.0,MASTERCARD
1,521714842,2020-08-31 12:41:23.219,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,45850,45850.0,MASTERCARD
2,521714218,2020-08-31 12:41:00.15,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,50876,50876.0,MASTERCARD
3,521713519,2020-08-31 12:40:19.128,8,839429,REFUND,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,DEBIT,500,500.0,VISA
4,521713519,2020-08-31 12:40:16.871,8,839429,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,DEBIT,500,500.0,VISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,US,DEBIT,24515,24515.0,MASTERCARD
570308,513712578,2020-08-11 00:13:54.683,8,839429,AUTHORIZATION_AND_CAPTURE,DECLINED,RESTRICTED_CARD,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,20050,20050.0,MASTERCARD
570309,513714958,2020-08-11 00:21:28.031,8,833883,AUTHORIZATION_AND_CAPTURE,APPROVED,APPROVED,CRED_MASTERCARD,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,65400,65400.0,MASTERCARD
570310,513712137,2020-08-11 00:48:47.001,8,833883,AUTHORIZATION_AND_CAPTURE,DECLINED,ANTIFRAUD_REJECTED,CRED_VISA,GATEWAY,POST_API,IMMEDIATE,CO,CREDIT,140875,140875.0,VISA


In [41]:
#Si bien se observa que Status cumple con esta característica de no superar más de 4 valores únicos, al ser una variable
#de importancia sobre las demás, pues a partir de ella se reconocé si la transacción es realizada o no, es mejor hacer una
#transformación que mantenga en una sola columna sus variaciones
daf6 = pd.get_dummies(daf5b, columns = ["Transaction type", "Status", "Payment model", "Transaction origin",
                                      "Accreditation model", "Card type"])
daf6

,Order Id,Creation date,mes,Account Id,Response code,Payment method,Country BIN ISO,Valor,Processing value,Franchise,...,Status_DECLINED,Payment model_AGGREGATOR,Payment model_GATEWAY,Transaction origin_POST_API,Transaction origin_STANDARD_HTML_v4_0,Accreditation model_IMMEDIATE,Accreditation model_Not specified Declined,Card type_CREDIT,Card type_DEBIT,Card type_Not specified
0,521715135,2020-08-31 12:41:45.866,8,833883,APPROVED,CRED_MASTERCARD,CO,34450,34450.0,MASTERCARD,...,0,0,1,1,0,1,0,1,0,0
1,521714842,2020-08-31 12:41:23.219,8,833883,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,CO,45850,45850.0,MASTERCARD,...,1,0,1,1,0,1,0,1,0,0
2,521714218,2020-08-31 12:41:00.15,8,833883,ANTIFRAUD_REJECTED,CRED_MASTERCARD,CO,50876,50876.0,MASTERCARD,...,1,0,1,1,0,1,0,1,0,0
3,521713519,2020-08-31 12:40:19.128,8,839429,APPROVED,CRED_VISA,CO,500,500.0,VISA,...,0,0,1,1,0,1,0,0,1,0
4,521713519,2020-08-31 12:40:16.871,8,839429,APPROVED,CRED_VISA,CO,500,500.0,VISA,...,0,0,1,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,ANTIFRAUD_REJECTED,CRED_MASTERCARD,US,24515,24515.0,MASTERCARD,...,1,0,1,1,0,1,0,0,1,0
570308,513712578,2020-08-11 00:13:54.683,8,839429,RESTRICTED_CARD,CRED_MASTERCARD,CO,20050,20050.0,MASTERCARD,...,1,0,1,1,0,1,0,1,0,0
570309,513714958,2020-08-11 00:21:28.031,8,833883,APPROVED,CRED_MASTERCARD,CO,65400,65400.0,MASTERCARD,...,0,0,1,1,0,1,0,1,0,0
570310,513712137,2020-08-11 00:48:47.001,8,833883,ANTIFRAUD_REJECTED,CRED_VISA,CO,140875,140875.0,VISA,...,1,0,1,1,0,1,0,1,0,0


Así con esta nueva transformación se pueden hacer otra limpieza necesaria, aquellas columnas a las que se les realizó rellenado de datos pérdidos en las que sea fácil identificar los otros valores originales, en este caso esta eliminación se basará en las columnas generadas a partir de Accreditation model y Card type pues es sencillo identificar aquellos datos específicos entre sus valores.
Adicionalmente se eliminarán las columnas que correspondan al segundo valor de las variables Payment model y Transaction origin, con el fin de evitar una correlación directa entre las 2 clumnas resultates de cada una. Así se entenderá que los valores 0 de la ahora variable Payment model_GATEWAY habrán sido transacciones de Payment model_AGGREGATOR y los valores 0 de la ahora variable Transaction origin_POST_API habrán sido transacciones de Transaction origin_STANDARD_HTML_v4_0

In [42]:
daf7 = daf6.drop(["Accreditation model_Not specified Declined", "Status_DECLINED", "Card type_Not specified", "Payment model_AGGREGATOR",
                 "Transaction origin_STANDARD_HTML_v4_0"], axis = 1)
daf7

,Order Id,Creation date,mes,Account Id,Response code,Payment method,Country BIN ISO,Valor,Processing value,Franchise,Transaction type_AUTHORIZATION_AND_CAPTURE,Transaction type_REFUND,Transaction type_VOID,Status_APPROVED,Payment model_GATEWAY,Transaction origin_POST_API,Accreditation model_IMMEDIATE,Card type_CREDIT,Card type_DEBIT
0,521715135,2020-08-31 12:41:45.866,8,833883,APPROVED,CRED_MASTERCARD,CO,34450,34450.0,MASTERCARD,1,0,0,1,1,1,1,1,0
1,521714842,2020-08-31 12:41:23.219,8,833883,INSUFFICIENT_FUNDS,REDEBAN_MASTERCARD,CO,45850,45850.0,MASTERCARD,1,0,0,0,1,1,1,1,0
2,521714218,2020-08-31 12:41:00.15,8,833883,ANTIFRAUD_REJECTED,CRED_MASTERCARD,CO,50876,50876.0,MASTERCARD,1,0,0,0,1,1,1,1,0
3,521713519,2020-08-31 12:40:19.128,8,839429,APPROVED,CRED_VISA,CO,500,500.0,VISA,0,1,0,1,1,1,1,0,1
4,521713519,2020-08-31 12:40:16.871,8,839429,APPROVED,CRED_VISA,CO,500,500.0,VISA,1,0,0,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,ANTIFRAUD_REJECTED,CRED_MASTERCARD,US,24515,24515.0,MASTERCARD,1,0,0,0,1,1,1,0,1
570308,513712578,2020-08-11 00:13:54.683,8,839429,RESTRICTED_CARD,CRED_MASTERCARD,CO,20050,20050.0,MASTERCARD,1,0,0,0,1,1,1,1,0
570309,513714958,2020-08-11 00:21:28.031,8,833883,APPROVED,CRED_MASTERCARD,CO,65400,65400.0,MASTERCARD,1,0,0,1,1,1,1,1,0
570310,513712137,2020-08-11 00:48:47.001,8,833883,ANTIFRAUD_REJECTED,CRED_VISA,CO,140875,140875.0,VISA,1,0,0,0,1,1,1,1,0


Ya bien estructuradas algunas columnas, aun falta transformar otras de tipo objeto también, en este caso para no ampliar en gran medida las variables a utilizar se buscará otra forma de hacer esta transformación

In [43]:
#Importando la librería de Sklearn bajo un codificador ordinal
from sklearn.preprocessing import OrdinalEncoder

In [44]:
#de antemano hay que recordar el orden de los datos únicos presentes en cada variable a transformar
daf7["Response code"].unique()

array(['APPROVED', 'INSUFFICIENT_FUNDS', 'ANTIFRAUD_REJECTED',
       'INVALID_TRANSACTION', 'ENTITY_DECLINED', 'RESTRICTED_CARD',
       'INVALID_EXPIRATION_DATE_OR_SECURITY_CODE',
       'PAYMENT_NETWORK_REJECTED', 'INVALID_CARD', 'EXPIRED_CARD',
       'REPEAT_TRANSACTION', 'EXCEEDED_AMOUNT',
       'INACTIVE_PAYMENT_PROVIDER', 'CONTACT_THE_ENTITY',
       'INTERNAL_PAYMENT_PROVIDER_ERROR', 'BANK_UNREACHABLE',
       'NOT_ACCEPTED_TRANSACTION', 'PAYMENT_NETWORK_NO_CONNECTION',
       'PAYMENT_NETWORK_BAD_RESPONSE', 'ENTITY_MESSAGING_ERROR',
       'FIX_NOT_REQUIRED', 'AUTOMATICALLY_FIXED_AND_SUCCESS_REVERSAL',
       'ERROR', 'PAYMENT_NETWORK_NO_RESPONSE'], dtype=object)

In [45]:
#Primero codificamos a través de la librería, a su vez se presentará el identificador correspondiente para cada valor
Enc1 = OrdinalEncoder(categories=[daf7["Response code"].unique()])
Enc1.fit(daf7[["Response code"]])
daf7["Response code"] = Enc1.transform(daf7[["Response code"]])
daf7["Response code"].unique() #cada valor corresponderá el valor de posición de la lista anterior, recordando que python
#mencionará la primera posición como 0

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [46]:
#Ahora la segunda variable a transformar de esta forma, tiene el siguiente orden
daf7["Payment method"].unique()

array(['CRED_MASTERCARD', 'REDEBAN_MASTERCARD', 'CRED_VISA',
       'CRED_DINERS', 'REDEBAN_VISA', 'CRED_AMEX', 'REDEBAN_DINERS',
       'REDEBAN_AMEX', 'PSE', 'CRED_VISA_DEBIT'], dtype=object)

In [47]:
Enc2 = OrdinalEncoder(categories=[daf7["Payment method"].unique()])
Enc2.fit(daf7[["Payment method"]])
daf7["Payment method"] = Enc2.transform(daf7[["Payment method"]])
daf7["Payment method"].unique()

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [48]:
#Continuando con el procedimiento
daf7["Country BIN ISO"].unique()

array(['CO', 'US', 'HK', 'Not specified', 'HN', 'BO', 'CL', 'BR', 'SV',
       'AR', 'TM', 'PT', 'RO', 'CH', 'TW', 'MX', 'IT', 'GB', 'CN', 'EC',
       'AU', 'VE', 'PE', 'PA', 'CA', 'UY', 'DO', 'DE', 'GT', 'BE', 'PR',
       'UA', 'ES', 'KY', 'KR', 'FI', 'MY', 'CR', 'IN', 'CW', 'RS', 'IE',
       'FR', 'JP', 'SG', 'SA', 'NO', 'DK', 'NI', 'GA', 'EG', 'IQ', 'ZA',
       'DZ', 'NZ', 'AT', 'NL', 'AE', 'GI', 'PH', 'MV', 'PL', 'IL', 'AD',
       'LB', 'UG', 'NG', 'EE', 'SK', 'QA', 'TR', 'KE', 'PF', 'BN', 'HU',
       'ID', 'BH', 'PY', 'BM', 'RU', 'TN', 'MA', 'MU', 'KH', 'AO', 'BD',
       'OM', 'TH', 'PK', 'BA', 'LV', 'SE', 'TT', 'CZ', 'GE', 'LC', 'PS',
       'AW', 'KG', 'BZ', 'MW', 'TZ', 'NP', 'SB', 'AG', 'LT', 'GR', 'ZW'],
      dtype=object)

In [49]:
Enc3 = OrdinalEncoder(categories=[daf6["Country BIN ISO"].unique()])
Enc3.fit(daf7[["Country BIN ISO"]])
daf7["Country BIN ISO"] = Enc3.transform(daf7[["Country BIN ISO"]])
daf7["Country BIN ISO"].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107.])

In [50]:
daf7["Franchise"].unique()

array(['MASTERCARD', 'VISA', 'DINERS', 'AMEX', 'PSE', 'VISA_DEBIT'],
      dtype=object)

In [51]:
Enc4 = OrdinalEncoder(categories=[daf7["Franchise"].unique()])
Enc4.fit(daf7[["Franchise"]])
daf7["Franchise"] = Enc4.transform(daf7[["Franchise"]])
daf7["Franchise"].unique()

array([0., 1., 2., 3., 4., 5.])

In [52]:
#Finalmente se convertirá la última variable tipo objeto que existe en el dataframe que es el creation date
daf7["Creation date"] = pd.to_datetime(daf7["Creation date"])

In [53]:
daf7

,Order Id,Creation date,mes,Account Id,Response code,Payment method,Country BIN ISO,Valor,Processing value,Franchise,Transaction type_AUTHORIZATION_AND_CAPTURE,Transaction type_REFUND,Transaction type_VOID,Status_APPROVED,Payment model_GATEWAY,Transaction origin_POST_API,Accreditation model_IMMEDIATE,Card type_CREDIT,Card type_DEBIT
0,521715135,2020-08-31 12:41:45.866,8,833883,0.0,0.0,0.0,34450,34450.0,0.0,1,0,0,1,1,1,1,1,0
1,521714842,2020-08-31 12:41:23.219,8,833883,1.0,1.0,0.0,45850,45850.0,0.0,1,0,0,0,1,1,1,1,0
2,521714218,2020-08-31 12:41:00.150,8,833883,2.0,0.0,0.0,50876,50876.0,0.0,1,0,0,0,1,1,1,1,0
3,521713519,2020-08-31 12:40:19.128,8,839429,0.0,2.0,0.0,500,500.0,1.0,0,1,0,1,1,1,1,0,1
4,521713519,2020-08-31 12:40:16.871,8,839429,0.0,2.0,0.0,500,500.0,1.0,1,0,0,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570307,513712424,2020-08-11 00:13:22.733,8,839429,2.0,0.0,1.0,24515,24515.0,0.0,1,0,0,0,1,1,1,0,1
570308,513712578,2020-08-11 00:13:54.683,8,839429,5.0,0.0,0.0,20050,20050.0,0.0,1,0,0,0,1,1,1,1,0
570309,513714958,2020-08-11 00:21:28.031,8,833883,0.0,0.0,0.0,65400,65400.0,0.0,1,0,0,1,1,1,1,1,0
570310,513712137,2020-08-11 00:48:47.001,8,833883,2.0,2.0,0.0,140875,140875.0,1.0,1,0,0,0,1,1,1,1,0


In [54]:
daf7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570015 entries, 0 to 570311
Data columns (total 19 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   Order Id                                    570015 non-null  int64         
 1   Creation date                               570015 non-null  datetime64[ns]
 2   mes                                         570015 non-null  int64         
 3   Account Id                                  570015 non-null  int64         
 4   Response code                               570015 non-null  float64       
 5   Payment method                              570015 non-null  float64       
 6   Country BIN ISO                             570015 non-null  float64       
 7   Valor                                       570015 non-null  int64         
 8   Processing value                            570015 non-null  float64      

In [55]:
#Llegados así al dataset limpio y requerido para el entrenamiento y tratamiento de ML
#daf7.to_excel("Atifraudfinal.xlsx")
#daf7.to_csv("Atifraudfinal.csv")